In [ ]:
!apt-get -qq update 
!apt-get -qq install imagemagick jq parallel
!pip install -q yq

In [ ]:
!cp -n /etc/ImageMagick-6/policy.xml /content/policy.xml
!cat /etc/ImageMagick-6/policy.xml \
| xq -x '(.policymap.policy[] | select(."@pattern" == "PDF") | ."@rights") |= "read|write"' \
> out.xml
!cp out.xml /etc/ImageMagick-6/policy.xml

In [ ]:
# Libraries you will find useful
import numpy as np
import scipy 
import skimage
import matplotlib.pyplot as plt
import cv2
from skimage import feature
from PIL import Image, ImageEnhance

In [ ]:
def num_file_in_dir(dir):
    n = !ls {dir} | wc -l
    return int(n[0])

In [ ]:
def img_from_pdf(pile):
    !cd "/content/"
    !rm -r {pile}_cap
    !mkdir -p {pile}_cap
    !convert "/content/drive/MyDrive/549/{pile}.pdf" {pile}_cap/{pile}-%02d.jpg
    # resize image
    %pushd {pile}_cap
    !ls *.jpg | parallel -j 20 convert -background black -gravity center -extent 50% {} {}
    %popd

In [ ]:
def show_bgr(img):
    s = 12
    plt.figure(figsize=(s,s))

    if len(img.shape) == 3:
        plt.imshow(img[:,:,::-1])
        return
    
    plt.imshow(img, cmap="gray")

def read_trace(trace_name, shape):
    t = cv2.imread(trace_name, 0) / 255
    t = cv2.resize(t, shape[::-1], interpolation=cv2.INTER_AREA)
    return t < 0.75

def process_outline(fname, shape):
    img = cv2.imread(fname)
    img = cv2.resize(img, shape[::-1], interpolation=cv2.INTER_AREA)

    only_red = (img[:,:,0] < 50) & (img[:,:,2] > 0)
    
    red_loc = np.argwhere(only_red)[:1]

    only_blue = (img[:,:,0] > 0) & (img[:,:,2] < 50)

    return (only_blue, tuple(red_loc[0]) if len(red_loc) > 0 else None)

def overlay_blue(img, mask, save_mask=None):

    m_i = np.zeros(img.shape)
    m_i[:,:,0] = mask * 1

    if save_mask != None:
        cv2.imwrite( save_mask, m_i * 255)

    return 0.8 * img + 0.2 * m_i

def create_annotations(pile, dry=1):

    im_shape = cv2.imread(f"/content/drive/MyDrive/549/proj/{pile}/ezgif-frame-001.jpg").shape[:2]
    print (im_shape)

    bottom, red_loc = process_outline(f"/content/{pile}_cap/{pile}-00.jpg", im_shape)

    if dry == 0:
        %mkdir -p "/content/drive/MyDrive/549/proj/masks/{pile}"
        %mkdir -p "/content/drive/MyDrive/549/proj/overlays/{pile}"

    for i in range(num_file_in_dir(f"/content/drive/MyDrive/549/proj/{pile}/")):
        adi = 1 + i
        img = cv2.imread(f"/content/drive/MyDrive/549/proj/{pile}/ezgif-frame-{adi:03d}.jpg", 1) / 255
        cap, _ = process_outline(f"/content/{pile}_cap/{pile}-{adi:02d}.jpg", im_shape)

        cap = scipy.ndimage.maximum_filter(cap * 1, 2) > 0

        shape = (bottom | cap) * 1
        filled = skimage.segmentation.flood_fill(shape, red_loc, 2)

        if dry == 1:
            fig, (a,b) = plt.subplots(1,2, figsize=(10,10))
            a.imshow(shape)
            b.imshow(filled)
            plt.show()
            if i == 2:
                break
        else:
            print (f"ezgif-frame-{adi:03d}.png")
            overlay = overlay_blue(img, filled > 1, save_mask=f"/content/drive/MyDrive/549/proj/masks/{pile}/ezgif-frame-{adi:03d}.png")
            cv2.imwrite(f"/content/drive/MyDrive/549/proj/overlays/{pile}/ezgif-frame-{adi:03d}.png", overlay * 255)

In [ ]:
pile = "pile21"
img_from_pdf(pile)
create_annotations(pile,0)

(480, 640)
ezgif-frame-001.png
ezgif-frame-002.png
ezgif-frame-003.png
ezgif-frame-004.png
ezgif-frame-005.png
ezgif-frame-006.png
ezgif-frame-007.png
ezgif-frame-008.png
ezgif-frame-009.png
ezgif-frame-010.png
ezgif-frame-011.png
ezgif-frame-012.png
ezgif-frame-013.png
ezgif-frame-014.png
ezgif-frame-015.png
ezgif-frame-016.png
ezgif-frame-017.png
ezgif-frame-018.png
ezgif-frame-019.png
ezgif-frame-020.png
ezgif-frame-021.png
ezgif-frame-022.png
ezgif-frame-023.png
ezgif-frame-024.png
ezgif-frame-025.png
ezgif-frame-026.png
ezgif-frame-027.png
ezgif-frame-028.png
ezgif-frame-029.png
ezgif-frame-030.png
ezgif-frame-031.png
ezgif-frame-032.png
ezgif-frame-033.png
ezgif-frame-034.png
ezgif-frame-035.png
ezgif-frame-036.png
ezgif-frame-037.png
ezgif-frame-038.png
